<a href="https://colab.research.google.com/github/davanstrien/people-iiif/blob/main/03_download_loc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install httpx loguru fastcore sparqlwrapper tqdm

     |████████████████████████████████| 81kB 4.4MB/s 
     |████████████████████████████████| 61kB 4.0MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 61kB 3.9MB/s 
     |████████████████████████████████| 235kB 23.5MB/s 
     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 51kB 3.6MB/s 


In [ ]:
import pandas as pd
import httpx
import io
from pathlib import Path
from fastcore.all import *
from PIL import Image
import hashlib
from loguru import logger

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
path = Path('/content/drive/MyDrive/IIIF_AI_project')

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/davanstrien/people-iiif/main/loc_harvester/image_by_subject.csv?token=ACEUI5LFQGQXOEYLLVXG5O3AT7A7O", index_col=0)

In [ ]:
df

,id,desc,term,source,url
0,tgm012273,"Library [of Congress, main reading room, Washi...",Glass,https://www.loc.gov/pictures/item/2016819413/,https://tile.loc.gov/storage-services/service/...
1,tgm012273,[Stained glass windows],Glass,https://www.loc.gov/pictures/item/2016819457/,https://tile.loc.gov/storage-services/service/...
2,tgm012273,Cone-shaped exterior feature at the Museum of ...,Glass,https://www.loc.gov/pictures/item/2018699148/,https://tile.loc.gov/storage-services/service/...
3,tgm012273,Cone-shaped exterior feature at the Museum of ...,Glass,https://www.loc.gov/pictures/item/2018699154/,https://tile.loc.gov/storage-services/service/...
4,tgm012273,"[Library of Congress (Washington, D.C.). Leade...",Glass,https://www.loc.gov/pictures/item/2001695198/,https://tile.loc.gov/storage-services/service/...
...,...,...,...,...,...
309,tgm005206,[Women in ball gowns and men in military unifo...,Hugging,https://www.loc.gov/pictures/item/2014636865/,https://tile.loc.gov/storage-services/service/...
310,tgm005206,"[Man wearing military uniform, holding rifle, ...",Hugging,https://www.loc.gov/pictures/item/2014636866/,https://tile.loc.gov/storage-services/service/...
311,tgm005206,[Hypnotist directing group of people to do unu...,Hugging,https://www.loc.gov/pictures/item/2014636869/,https://tile.loc.gov/storage-services/service/...
312,tgm005206,Bronson Howard's greater Shenandoah,Hugging,https://www.loc.gov/pictures/item/2014637152/,https://tile.loc.gov/storage-services/service/...


In [ ]:
df['urlhash'] = pd.util.hash_pandas_object(df.url, index=False)

In [ ]:
Path(path/'loc-images-labels').mkdir(exist_ok=True)
save_path = Path(path/'loc-images-labels')

In [ ]:
def download_image(row, path=save_path):
    url = row.url
    hash = row.urlhash
    ext = url.split('.')[-1]
    fpath = f'{path}/{hash}.{ext}'
    if not Path(fpath).is_file():
        try:
            r = httpx.get(url, timeout=30)
            im = Image.open(io.BytesIO(r.content))
            im.save(fpath)
            return fpath         
        except Exception as e:
            logger.exception(e)
    else:
        pass

In [ ]:
df['dl_fpath'] = df.progress_apply(download_image, axis=1)

In [ ]:
df.to_csv(path/'loc-images-labels.csv')